In [52]:
from math import ceil, floor
import random
import time

#random.seed(4371)
def gen_board_solved(size):
    '''
    :Function that generates a board
    :using the knights move to place
    :all the queens, by splitting the board in 2
    :halves, resulting in a solved board for all
    :cases where size % 6 != 2 or 3
    :Complexity is O(n)
    '''
    qRow = [0 for i in range(size)]                    #qRow[i] is number of queen on i-th row
    qCol = [0 for i in range(size)]                    #qCol[i] is row of queen in i-th col
    qMainDiag = [0 for i in range(2*size-1)]           #qMainDiag[i] is number of queens on i-th main diagonal
    qSecDiag = [0 for i in range(2*size-1)]            #qSecDiag[i] is number of queens on i-th secondary diagonal
    for i in range(size):
        if i < ceil(size/2):                       #first we place queen with knight's step on bottom half of board
            qRow[floor(size/2)+i]+=1                   #qRow[row] = #queens on row 'row'
            qCol[2*i]=floor(size/2)+i                  #qCol[col] = row : queen on 'col'-th col is on 'row'-th row
            qMainDiag[i - floor(size/2)+size-1]+=1        #index = col-row+size-1 : queen is on index-th main diagonal
            qSecDiag[floor(size/2)+3*i]+=1             #index = col+row     : queen is on index-th secondary diagonal
        else:                                      #then we place the rest of the queens on upper half of board again using knight's step
            qRow[i-ceil(size/2)]+=1                    #qRow[row] = #queens on row y
            qCol[1+(i-ceil(size/2))*2]=i-ceil(size/2)  #qCol[col] = row : queen on x-th col is on y-th row
            qMainDiag[i-ceil(size/2)+size]+=1          #index = col-row+size-1 : queen is on index-th main diagonal
            qSecDiag[1+(i-ceil(size/2))*3]+=1          #index = col+row     : queen is on index-th secondary diagonal
    # print(qCol)
    # print(qMainDiag,sum(qMainDiag))
    # print(qSecDiag,sum(qSecDiag))
    return qRow,qCol,qMainDiag,qSecDiag

In [56]:
def get_row_with_min_value(col_index,qRow,qCol,qMainDiag,qSecDiag,size):
    '''
    :Function that takes a column index
    :and returns the row with least amount
    :of conflicts for that column's queen
    :Complexity is O(n)
    '''
    #print(f'row : {qCol[col_index]} col: {col_index}')
    min_conflicts = size
    index_list = []
    for row_index in range(size):
        if (row_index == qCol[col_index]):
            continue
        #We fetch info about each cell using its corresponding row, main diagonal and secondary diagonal
        conflicts_count = qRow[row_index] + qMainDiag[col_index-row_index+size-1] + qSecDiag[col_index+row_index]
        #print(f'row:{row_index} has {conflicts_count} conflicts')
        if(conflicts_count == min_conflicts):
            # print(f'row:{row_index} has {conflicts_count} conflicts')
            index_list.append(row_index)
        elif(conflicts_count<min_conflicts) :
            # print(f'row:{row_index} has {conflicts_count} conflicts')
            min_conflicts= conflicts_count
            index_list.clear()
            index_list.append(row_index)
        else: continue
    return random.choice(index_list)


In [43]:
def gen_board(size):
    '''
    :Function that generates board of size n
    :Using random allocation for each queen
    :Returns a tuple of following variables\:
    :qRow - list with number of queens on i-th row
    :qCol - list with row positions for queens on each column
    :qMainDiag - list with number of queens on i-th main diagonal
    :aSecDiag - list with number of queens on i-th secondary diagonal
    :Complexity is O(n)
    '''
    qRow = [0 for i in range(size)]                    #qRow[i] is number of queen on i-th row
    qCol = [0 for i in range(size)]                    #qCol[i] is row of queen on i-th col
    qMainDiag = [0 for i in range(2*size-1)]           #qMainDiag[i] is number of queens on i-th main diagonal
    qSecDiag = [0 for i in range(2*size-1)]            #qSecDiag[i] is number of queens on i-th secondary diagonal
    rows_perm = list(range(0,size))                    #get a random permutation of the rows
    random.shuffle(rows_perm)
    for col_index in range(size):
        #row_index = get_row_with_min_value(col_index,qRow,qCol,qMainDiag,qSecDiag,size)
        row_index = rows_perm[col_index]
        qRow[row_index]+=1
        qCol[col_index] = row_index
        qMainDiag[col_index-row_index+size-1] +=1
        qSecDiag[col_index+row_index] +=1
    return qRow,qCol,qMainDiag,qSecDiag


In [44]:
def print_board(qCol,size):
    '''
    :Function that prints the board
    '''
    for row in range(size):
        print('_ '*qCol[row] + '*' + ' _'*(size-qCol[row]-1))


In [45]:

def conflicts(qRow,qCol,qMainDiag,qSecDiag,size):
    '''
    :Function that takes information about the board
    :and returns a dictionary where the key is
    :column index and value is its respective
    :queen's conflicts
    :Complexity is O(n)'''
    conflicts_list = {}
    for col_index in range(size):
        conflicts_count = 0
        row_index = qCol[col_index]
        main_diag_i = col_index-row_index+size-1                        #index = col_num - row_num + size - 1
        sec_diag_i = col_index+row_index                               #index = col_num + row_num
        # if >1 on the same row or >1 on the same main diag or >on the same sec diag
        if(qRow[row_index] > 1 ) : conflicts_count+=qRow[row_index]-1
        if(qMainDiag[main_diag_i]>1) : conflicts_count+= qMainDiag[main_diag_i] - 1
        if(qSecDiag[sec_diag_i]>1) : conflicts_count+=qSecDiag[sec_diag_i] - 1
        conflicts_list[col_index]=conflicts_count
        # print(f'queen on the {col_index}-th col collides with {qRow[qCol[col_index]]-1} queens on its row, {qMainDiag[main_diag_i]-1} queens on its main diagonal and {qSecDiag[sec_diag_i]-1} queens on its second diagonal')
    return conflicts_list

In [46]:
def move_queen_to_new_cell(qRow,qCol,qMainDiag,qSecDiag,size,queen_col_index,queen_new_row):
    '''
    :Function that moves the queen
    :on col_index column to a new row
    :and returns updated conflicts list
    :Complexity is O(n)
    '''
    #move queen to new row and update conflicts_list
    queen_old_row = qCol[queen_col_index]
    qRow[queen_old_row]-=1                                  #remove queen from old row
    qMainDiag[queen_col_index-queen_old_row+size-1]-=1      #remove queen from old main diag
    qSecDiag[queen_col_index+queen_old_row]-=1              #remove queen from old sec diag
    qCol[queen_col_index] = queen_new_row                   #move queen to new row
    qRow[queen_new_row]+=1                                  #update number of queen on new row
    qMainDiag[queen_col_index-queen_new_row+size-1]+=1      #add queen to new main diag
    qSecDiag[queen_col_index+queen_new_row]+=1              #add queen to new sec diag
    #update conflicts_list
    return conflicts(qRow,qCol,qMainDiag,qSecDiag,size)
    

In [47]:
def get_queen_with_max_conflicts(conflicts_list):
    '''
    :Function that returns column index of queen
    :with maximun conflicts, or a random queen
    :in case of multiple indices
    :Complexity is O(n)
    '''
    max_conflicts = 0
    index_list = []
    for key in conflicts_list:
        if(conflicts_list[key] == max_conflicts):
            index_list.append(key)
        elif(conflicts_list[key] > max_conflicts):
            max_conflicts = conflicts_list[key]
            index_list = [key]
        else : continue
    return random.choice(index_list)

In [48]:
# #ORIGINAL
# def solve(n) :
#     if(n%6 != 2 and n%6 != 3):
#         qRow,qCol,qMainDiag,qSecDiag = gen_board_solved(n)
#         #print_board(qCol,n)
#         print(True)
#         print(f'Try number {0}----------------------------------------------------------')
#         return
#     else : 
#         qRow,qCol,qMainDiag,qSecDiag = gen_board(n)
#     conflicts_list = conflicts(qRow,qCol,qMainDiag,qSecDiag,n)
#     i = 0
#     conflicts_count=1
#     while(conflicts_count !=0):
#         # print_board(qCol,n)
#         conflicts_count = 0
#         for key in conflicts_list:
#             conflicts_count+= conflicts_list[key]
#         # print(f'number of conflicts {conflicts_count}')
#         if(conflicts_count == 0) : 
#             print(True)
#             print(f'Try number {i}----------------------------------------------------------')
#             return
#         col = get_queen_with_max_conflicts(conflicts_list)
#         # print(f'col to be moved: {col}')
#         queen_new_row = get_row_with_min_value(col,qRow,qCol,qMainDiag,qSecDiag,n)
#         #print(f'i choose {queen_new_row}th row')
#         #print(f'old conflict list is {conflicts_list}')
#         conflicts_list = move_queen_to_new_cell(qRow,qCol,qMainDiag,qSecDiag,n,col,queen_new_row)
#         # print(f'new conflict list is {conflicts_list}')
#         #print_board(qCol,n)
#         i+=1
#     print(False)
#     return
#     # print(f'new conflict list is {conflicts_list}')



In [68]:
#FINAL
def solve(n:int) :
    '''
    :Solving the N-Queens problem using minConflicts algorithm
    '''
    if(n%6 != 2 and n%6 != 3):
        qRow,qCol,qMainDiag,qSecDiag = gen_board_solved(n)
        # print_board(qCol,n)
        # print(True)
        # print(f'Try number 0')
        return qCol
    else : 
        qRow,qCol,qMainDiag,qSecDiag = gen_board(n)
    conflicts_list = conflicts(qRow,qCol,qMainDiag,qSecDiag,n)
    i = 0
    conflicts_count=1
    while(conflicts_count !=0 and i<n**2):
        conflicts_count = 0
        #O(n) complexity
        for key in conflicts_list:
            conflicts_count+= conflicts_list[key]
        if(conflicts_count == 0) :  
            # print_board(qCol,n)
            return  qCol
        col = get_queen_with_max_conflicts(conflicts_list)
        queen_new_row = get_row_with_min_value(col,qRow,qCol,qMainDiag,qSecDiag,n)
        conflicts_list = move_queen_to_new_cell(qRow,qCol,qMainDiag,qSecDiag,n,col,queen_new_row)
        i+=1  
    # print(False)
    return  []



In [81]:

n = int(input())
start = time.time()
solution= solve(n)
finish = time.time()
print_board(solution,n)
print(f'it took {finish-start} seconds to find a solution')


_ _ _ * _ _ _ _ _
_ _ _ _ _ * _ _ _
* _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ *
_ _ _ _ * _ _ _ _
_ _ _ _ _ _ _ * _
_ * _ _ _ _ _ _ _
_ _ _ _ _ _ * _ _
_ _ * _ _ _ _ _ _
it took 0.0 seconds to find a solution


time to generate the board is 0.012964010238647461sec
time to generate conflicts is 0.004987239837646484sec
average time to check conflicts is 0.6813815033319096ms
average time to find worst queens is 1.0420789438487597ms
average time to find optimal row  is 2.8383597757778602ms
average time to move queen is 1.0420789438487597ms
True
Try number 3983
it took 35.07129168510437 seconds to find a solution